# Experiment 2
### In this experiment we compare different metamodels on an image completion task framed as a 2D regression task 
We compare the amortised GI BNN with a convolutional conditional neural process and an amortised MFVI BNN as two baseline models. The models will be evaluated on datasets which will be treated as metadatasets in which each image will be subsampled and treated as a 2D regression dataset. The following datasets will be considered:
- MNIST
- CelebA, including an out-of-distribution qualitative evaluation on the Ellen Oscars Selfie
- CIFAR10

For each test case, a linear interpolator will be used as a lower benchmark


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import tqdm.auto as tqdm
import sys

torch.set_default_dtype(torch.float64)

sys.path.append("../")
from bnn_amort_inf.models.bnn import gibnn, mfvi_bnn
from bnn_amort_inf.models.np import GridConvCNP, CNP
from bnn_amort_inf import utils

### Handle Data and Generate Metadatasets

In [ ]:
num_datasets = 200  # note cannot be greater than 9000
meta_datasets = {}
ratio = 0.1  # proportion of context pixels in image

mnist = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)
mnist_iter = iter(torch.utils.data.DataLoader(mnist, shuffle=True))
img_samp = next(mnist_iter)[0]

I, M_c = (
    next(mnist_iter)[0].squeeze(0),
    utils.dataset_utils.random_mask(ratio, img_samp)[1],
)

meta_datasets["mnist"] = utils.dataset_utils.MetaDataset(
    [
        (
            next(mnist_iter)[0].squeeze(0),
            utils.dataset_utils.random_mask(ratio, img_samp)[1],
        )
        for _ in range(num_datasets)
    ]
)

### Define and Train Models

In [ ]:
plot_training_metrics = True

In [ ]:
convcnp = GridConvCNP(
    x_dim=2,
    y_dim=1,
    embedded_dim=64,
    cnn_chans=[64, 64],
    kernel_size=9,
)

convcnp_tracker = utils.training_utils.train_metamodel(
    convcnp,
    meta_datasets["mnist"],
    gridconv=True,
    lr=1e-3,
    max_iters=50_000,
    batch_size=10,
    min_es_iters=10_000,
    ref_es_iters=1_000,
    smooth_es_iters=500,
)

if plot_training_metrics:
    fig, axes = plt.subplots(
        len(convcnp_tracker.keys()),
        1,
        figsize=(8, len(convcnp_tracker.keys()) * 4),
        dpi=100,
        sharex=True,
    )

    for ax, (key, vals) in zip(axes, convcnp_tracker.items()):
        ax.plot(vals)
        ax.set_ylabel(key)
        ax.grid()

    plt.show()